# Домашнее задание - линейная регрессия

### Работа с признаками (8 баллов)

Скачайте датасет из материалов к уроку или по ссылке https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv 


Описание признаков:

* Agency — название страхового агентства
* Agency Type — тип страхового агентства
* Distribution Channel — канал продвижения страхового агентства
* Product Name — название страхового продукта
* Duration — длительность поездки (количество дней)
* Destination — направление поездки
* Net Sales — сумма продаж 
* Commission (in value) — комиссия страхового агентства
* Gender — пол застрахованного
* Age — возраст застрахованного

Ответ:
* Claim — потребовалась ли страховая выплата: «да» — 1, «нет» — 0

Обработайте пропущенные значения и примените написанные функции onehot_encode() и minmax_scale().

**Подсказка**: маску для категориальных признаков можно сделать фильтром cat_features_mask = (df.dtypes == "object").values

In [31]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


def minmax_scale(matrix: np.array, f_range=(0, 1)) -> np.array:
    axis = 0 if len(matrix) > 1 else None
    d_part = matrix.max(axis=axis) - matrix.min(axis=axis)
    if matrix.size == 1:
        d_part = 1
    matrix_std = (matrix - matrix.min(axis=axis)) / d_part

    matrix_scaled = matrix_std * (f_range[1] - f_range[0]) + f_range[0]
    return matrix_scaled


def onehot_encoding(matrix: np.array) -> np.array:
    result = np.zeros([len(matrix), len(set(matrix))], dtype=int)
    dictionary_values = {index: value for value, index in enumerate(sorted(set(matrix)))}
    for value, index in enumerate(matrix):
        result[value][dictionary_values[index]] = 1
    return result


dataframe = pd.read_csv('travel insurance.csv')
cat_features_mask = (dataframe.dtypes == "object").values
replacer = SimpleImputer(strategy="mean")
scaled_data = minmax_scale(pd.DataFrame(data=replacer.fit_transform(dataframe[dataframe.columns[~cat_features_mask]]),
                                       columns=dataframe[dataframe.columns[~cat_features_mask]].columns))
result_dict = {}
for feature in dataframe[dataframe.columns[cat_features_mask]].fillna(""):
    if feature != 'Claim':
        for value, index in enumerate(onehot_encoding(dataframe[dataframe.columns[cat_features_mask]].fillna("")[feature]).T):
            result_dict[feature + str(value + 1)] = index

cleared_data = pd.concat([scaled_data, pd.DataFrame(result_dict)], axis=1)
print(cleared_data)

       Duration  Net Sales  Commision (in value)       Age  Agency1  Agency2  \
0      0.038501   0.300250              0.033757  0.686441        0        0   
1      0.038501   0.300250              0.033757  0.601695        0        0   
2      0.013721   0.283153              0.104762  0.271186        0        0   
3      0.012697   0.291410              0.083810  0.271186        0        0   
4      0.016588   0.307923              0.041905  0.347458        0        0   
...         ...        ...                   ...       ...      ...      ...   
63321  0.023142   0.353628              0.043210  0.262712        0        0   
63322  0.012288   0.357798              0.049383  0.338983        0        0   
63323  0.000819   0.339450              0.022222  0.483051        0        0   
63324  0.001024   0.339450              0.022222  0.533898        0        0   
63325  0.004915   0.346122              0.032099  0.296610        0        0   

       Agency3  Agency4  Agency5  Agenc

# Подробный анализ и подготовка датасета часто помогают улучшить качество модели. Ниже представлено несколько идей преобразований. Вы можете применить одно или несколько из этих преобразований (а можете не применять), чтобы помочь будущей модели. 

1. Посмотрите на количественные признаки. Возможно, в некоторых признаках есть выбросы - значения, которые сильно выбиваются. Такие значения полезно удалять. Советуем присмотреться к колонке Duration)

2. Можно заметить, что one hot encoding сильно раздувает количество столбцов. Радикальное решение - можно попробовать выбросить все категориальные признаки из датасета.

3. Если все-таки оставляете категориальные признаки, то подумайте, как уменьшить количество столбцов после one hot encoding. Признаки с большим количеством значений (Duration - 149! разных стран) можно удалить или попробовать сгруппировать некоторые значения.

4. Downsampling. Датасет достаточно большой, разница в классах огромная. Можно уменьшить число наблюдений с частым ответом.

In [42]:
cleared_data = scaled_data # Использование только не категориальных фич

### Применение линейной регрессии (10 баллов)

Это задача классификации, но её можно решить с помощью линейной регрессии, если округлять предсказанный ответ до целого и выбирать ближайший по значению ответ из множества {0, 1}.

Вынесите признак 'Claim' в вектор ответов и разделите датасет на обучающую и тестовую выборку в соотношении 80 к 20. Зафиксируйте random_state.

**Подсказка:** быстро перевести Yes/No в 1/0 можно так - np.where(df['Claim'] == 'Yes', 1,0)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(np.array(cleared_data),
                                                    np.where(dataframe[dataframe.columns[cat_features_mask]].fillna("")['Claim'] == 'Yes',
                                                            1, 
                                                            0),
                                                    train_size=0.8,
                                                    random_state=143)


Найдите аналитическое решение для обучающей выборки: обычное и регуляризацией l2. 

In [44]:
X_train_2 = np.hstack((np.ones((np.array(X_train.shape)[0], 1)), X_train))
analytic_solved = np.dot(np.dot(np.linalg.inv(X_train_2.T @ X_train_2), X_train_2.T), y_train)
print(analytic_solved)

[-0.10402076  0.08705588  0.33922122  0.03308683 -0.01465457]


In [45]:
X_train_2 = np.hstack((np.ones((np.array(X_train.shape)[0], 1)), X_train))
lambda_ = 1
analytic_solved_2 = np.linalg.inv(X_train_2.T @ X_train_2 + lambda_ * lambda_ * np.eye(k + 1)) @ X_train_2.T @ y_train
print(analytic_solved_2)

[-0.10148202  0.08608959  0.33202727  0.03577968 -0.01477802]


Постройте модель LinearRegression, примените к тестовой выборке и посчитайте MSE (можно использовать библиотеку sklearn)

In [46]:
sklearn_linear_regressor = LinearRegression()
sklearn_linear_regressor.fit(X_train, y_train)

LinearRegression()

In [47]:
print(mean_squared_error(y_test, sklearn_linear_regressor.predict(X_test)))

0.01417171759587896


### Вывод (1 балла)

Напишите краткий вывод по заданию (достаточно пары предложений). Расскажите, какие способы предобработки данных вы выбрали и почему. Насколько хороша ваша модель?

Вывод: При подготовке датасета я выбрал второй предложенный пример,
так как мои предварительные исследования показали,
что удаление всех категориальных признаков значительно снижает MSE.
В этом датасете было огромное количество категориальных признаков, которые могут повлиять на правильную работу модели.
Кроме того, я заметил, что аналитическое решение регрессии практически не отличается от решения с использованием регуляризации.

Таким образом, мои результаты подтверждают важность правильной предобработки данных перед обучением модели и использования регуляризации для предотвращения переобучения. В будущем, я буду учитывать эти факторы при работе с подобными датасетами и принимать во внимание возможность удаления категориальных признаков для улучшения результатов модели.